In [1]:
import os.path as path
from pyspark.sql import SparkSession
from pyspark.sql import column
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark.ml.feature import StringIndexer, VectorAssembler,OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier

In [2]:
def init_spark():
  return SparkSession \
        .builder \
        .appName("Big data project") \
        .config("spark.executor.memory", "8g") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()

spark = init_spark()

24/03/20 17:13:55 WARN Utils: Your hostname, DESKTOP-968IKC4 resolves to a loopback address: 127.0.1.1; using 172.31.102.226 instead (on interface eth0)
24/03/20 17:13:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 17:13:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [35]:
def get_data_dataframe(dir="final_dataset.csv"):
    # Specify the directory where the CSV files are saved
    csv_directory = "final_dataset.csv"
    
    # Read the CSV files back into a DataFrame
    final_df_read = spark.read.option("header", "true").csv(csv_directory)
    
    # Add a random column to shuffle data randomly
    shuffled_df = final_df_read.withColumn("rand", rand())
    shuffled_df = shuffled_df.orderBy("rand")
    final_df_read = shuffled_df.drop("rand")
    return final_df_read.limit(40000)



In [36]:
def print_class_distribution(df):
    # Group the DataFrame by the "genre" column and count the occurrences of each genre
    genre_counts = df.groupBy("genre").count()
    
    # Show the genre counts
    genre_counts.show()

# Replacing Null Values With Zero

In [37]:
def show_null_values_distribution(train_df):
    null_value_list = list()
    for col_ in train_df.columns:
        null_value_list.append(train_df.filter(train_df[col_]=="\\N").count())                     
    plt.rcParams["figure.figsize"] = (10,6)
    columns = [col_ for col_ in train_df.columns]
    
    plt.figure(figsize=(10, 6))
    plt.bar(columns, null_value_list, color='skyblue')
    plt.xlabel('Columns')
    plt.ylabel('Number of Null Values')
    plt.ylim(0, train_df.count())
    plt.title('Distribution of Null Values in Columns')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Drop Duplicates

In [38]:
def drop_duplicates(train_df):
    print(train_df.count())
    train_df=train_df.dropDuplicates()
    print(train_df.count())

# Normalization

In [39]:
def get_stages_of_normalization(categoricalColumns =['title',
     'director',
     'writer',
     'actorPrimaryName'], predicting='genre'):
    stages = []
    for categoricalCol in categoricalColumns:
        stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
        # encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
        stages += [stringIndexer]
    label_stringIdx = StringIndexer(inputCol = predicting, outputCol = 'label')
    stages += [label_stringIdx]
    
    assemblerInputs = [c + "Index" for c in categoricalColumns] 
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]
    return stages

In [40]:

def get_normalized_data(df):
    pipeline = Pipeline(stages = stages)
    pipelineModel = pipeline.fit(df)
    final_df_read = pipelineModel.transform(df)
    selectedCols = ['label', 'features','genre']+['title',
     'director',
     'writer',
     'actorPrimaryName']
    final_df_read = final_df_read.select(selectedCols)
    final_df_read.printSchema()
    return final_df_read

# Utils

In [41]:
def print_and_return_mapping_of_index_to_label(df,predicting='genre'):
    # Collect distinct pairs of (label, genre)
    label_genre_mapping = df.select("label", predicting).distinct().collect()
    map={}
    # Print the mapping
    for mapping in label_genre_mapping:
        print("Label %s is mapped to genre '%s'" % (mapping.label, mapping.genre))
        map[mapping.label]= mapping.genre
    return map

# Train and Test

In [42]:
final_df_read=get_data_dataframe()



In [43]:
stages=get_stages_of_normalization()
final_df_read=get_normalized_data(final_df_read)

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- genre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- actorPrimaryName: string (nullable = true)



In [44]:
# Split the data into training and testing sets
train_df, test_df = final_df_read.randomSplit([0.8, 0.2], seed=42)

# Show the number of rows in each set
print("Training set count:", train_df.count())
print("Testing set count:", test_df.count())

24/03/20 17:23:24 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


Training set count: 32011


24/03/20 17:23:25 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


Testing set count: 7989


In [45]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxBins=60000)
dtModel = dt.fit(train_df)

24/03/20 17:23:27 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/03/20 17:23:27 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/03/20 17:23:27 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 60000 to 32011 (= number of training instances)
24/03/20 17:23:27 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/03/20 17:23:31 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/03/20 17:23:38 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
24/03/20 17:23:51 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/03/20 17:24:16 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


In [46]:
predictions = dtModel.transform(test_df)
predictions.show(10)

24/03/20 17:25:32 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


+-----+--------------------+-----+--------+------------------+------------------+--------------------+--------------------+--------------------+----------+
|label|            features|genre|   title|          director|            writer|    actorPrimaryName|       rawPrediction|         probability|prediction|
+-----+--------------------+-----+--------+------------------+------------------+--------------------+--------------------+--------------------+----------+
|  0.0|[1.0,2234.0,2010....|Drama|    Vera|  Nedeljko Kovacic| Kristina Djukovic|         Nikola Kojo|[221.0,23.0,67.0,...|[0.68847352024922...|       0.0|
|  0.0|[1.0,4372.0,4393....|Drama|    Vera|   Steffy Argelich|   Steffy Argelich|        John Steiner|[221.0,23.0,67.0,...|[0.68847352024922...|       0.0|
|  0.0|[1.0,4372.0,4393....|Drama|    Vera|   Steffy Argelich|   Steffy Argelich|María Jose Gil Go...|[221.0,23.0,67.0,...|[0.68847352024922...|       0.0|
|  0.0|[2.0,2266.0,2216....|Drama| Breathe|      Onur Karaman|  

# Evaluation

In [52]:


def print_statistis(predictions,mapping):

    # Compute raw scores on the test set
    predictionAndLabels = predictions.rdd.map(lambda lp: (lp.prediction, lp.label))
    
    # Instantiate metrics object
    metrics = MulticlassMetrics(predictionAndLabels)
    confusion_matrix = metrics.confusionMatrix().toArray()
    
    # Overall statistics
    precision = metrics.precision(1.0)
    recall = metrics.recall(1.0)
    f1Score = metrics.fMeasure(1.0)
    total_predictions = confusion_matrix.sum(axis=1)
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    
    # Statistics by class
    labels = predictions.rdd.map(lambda lp: lp.label).distinct().collect()
    accuracies = {}
    for label in sorted(labels):
        print("______________________"+mapping[label]+"_____________________")
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))
        
    
    # Calculate the accuracy for each label
    print(total_predictions)
    for label in range(len(total_predictions)):
        correct_predictions = confusion_matrix[label, label]
        accuracy = correct_predictions / total_predictions[label]
        accuracies[label] = accuracy
    
    # Print accuracies for each label
    for label, accuracy in accuracies.items():
        print("Accuracy for label %s: %s" % (label, accuracy))


In [53]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

24/03/20 17:29:54 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


Test Accuracy: 0.40


In [54]:
mapp=print_and_return_mapping_of_index_to_label(train_df)

Label 0.0 is mapped to genre 'Drama'
Label 1.0 is mapped to genre 'Comedy'
Label 2.0 is mapped to genre 'Action'
Label 3.0 is mapped to genre 'Horror'
Label 4.0 is mapped to genre 'Thriller'
Label 5.0 is mapped to genre 'Romance'


24/03/20 17:29:57 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


In [55]:
print_statistis(predictions,mapp)

/home/umang/.local/lib/python3.10/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
24/03/20 17:29:58 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
24/03/20 17:29:58 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
24/03/20 17:31:03 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


Summary Stats
Precision = 0.3426724137931034
Recall = 0.14058355437665782
F1 Score = 0.19937304075235107


______________________Drama_____________________
Class 0.0 precision = 0.4054972661445249
Class 0.0 recall = 0.9137529137529138
Class 0.0 F1 Measure = 0.5617195496417605
______________________Comedy_____________________
Class 1.0 precision = 0.3426724137931034
Class 1.0 recall = 0.14058355437665782
Class 1.0 F1 Measure = 0.19937304075235107
______________________Action_____________________
Class 2.0 precision = 0.5238095238095238
Class 2.0 recall = 0.11412609736632083
Class 2.0 F1 Measure = 0.1874180865006553
______________________Horror_____________________
Class 3.0 precision = 0.7619047619047619
Class 3.0 recall = 0.02069857697283312
Class 3.0 F1 Measure = 0.04030226700251889
______________________Thriller_____________________
Class 4.0 precision = 0.0
Class 4.0 recall = 0.0
Class 4.0 F1 Measure = 0.0
______________________Romance_____________________
Class 5.0 precision = 0.0
Class 5.0 recall = 0.0
Class 5.0 F1 Measure = 0.0
[3003. 2262. 1253.  773.  545.  153.]
Accuracy for label 